# OpenAI Experimentation

The purpose of this notebook is to try out the different OpenAI features to familiarise myself with them, with the ultimate aim of using GPT-3 for Named Entity Recognition to anonymise our care notes reliably for analytics.

The features tested will be:
- Completion: Training GPT-3 to response to prompts to perform useful tasks

https://beta.openai.com/docs/guides/completion/introduction

## Setup

This step contains any imports and common setup functions

In [2]:
import os
import openai

from helpers import OPENAI_API_KEY

In [3]:
openai.api_key = OPENAI_API_KEY

## Completion

### Testing

In [72]:
phrase = 'Are you a real person'

response = openai.Completion.create(engine="davinci", prompt=phrase, max_tokens=50)

print(response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "? (NOT FOR OLDER ACTIVES :-)\n\nLooking for fun, mein Frau Geburtstag ist am 12.11.15\n\nSchert Euch blo\u00df Metzger inen blasen"
    }
  ],
  "created": 1643296365,
  "id": "cmpl-4V0k1Cmr0L4HygRrHRqzxXBiMqkyl",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


In [11]:
song = '''There was once a brave young girl named Callie. She lived in a middle-class neighbourhood in Glasgow, 
so life had never been tougher. 

One day she'''

song_response = openai.Completion.create(engine="davinci", prompt=song, max_tokens=100)

print(song_response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " was really bored, so she got all of Mum's pruning shears and the meat cleaver. When her Mum was out playing the bagpipes, she thought she would have some FUN!\n\nFirst of all Callie decided to play with that pesky next door neighbour, Phil. He annoyed her with his annoying yells and how he always envied Callie's youth and freedom. So she sneaked into his flat and came out with a big smile. He wasn't as deep fried"
    }
  ],
  "created": 1642581780,
  "id": "cmpl-4S0qSL8NRw3d3mUygiWvnuP8r0Xc5",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


In [12]:
question = '''There was once a brave young girl named Callie. She lived in a middle-class neighbourhood in Glasgow, 
so life had never been tougher. 

Where did Callie live?'''

question_response = openai.Completion.create(engine="davinci", prompt=question, max_tokens=100)

print(question_response)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " In a two-up two-down in Glasgow. It was very special because the area used to be pretty posh. She didn't have parquet. She had chipped and cracked linoleum. And no fancy gateway like Scotty had. She didn't have open-topped sports cars that her mates could jump from into a swimming pool. If a fire broke out she couldn't just carry oxygen tanks and a fire blanket as she ran back inside. What she had was a H"
    }
  ],
  "created": 1642581824,
  "id": "cmpl-4S0rAUfQPkc8KU0pad2Sv7ijPQtJN",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}


In [13]:
print(question + question_response['choices'][0]['text'])

There was once a brave young girl named Callie. She lived in a middle-class neighbourhood in Glasgow, 
so life had never been tougher. 

Where did Callie live? In a two-up two-down in Glasgow. It was very special because the area used to be pretty posh. She didn't have parquet. She had chipped and cracked linoleum. And no fancy gateway like Scotty had. She didn't have open-topped sports cars that her mates could jump from into a swimming pool. If a fire broke out she couldn't just carry oxygen tanks and a fire blanket as she ran back inside. What she had was a H


### ChatBot

In [42]:
conversation = '''Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: No
                  Bot:'''

In [43]:
chat_response = openai.Completion.create(engine="davinci", prompt=conversation, max_tokens=50)

chat_response

<OpenAIObject text_completion id=cmpl-4SmMHK5DDbGbLiTYdHFukBTyKaz8V at 0x10cfca290> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " OK, I'm sorry for the inconvenience. Goodbye.\n                 \n\n\nCustomer: Hi, I'm looking for a book.\n       "
    }
  ],
  "created": 1642764421,
  "id": "cmpl-4SmMHK5DDbGbLiTYdHFukBTyKaz8V",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}

In [10]:
print(conversation + chat_response['choices'][0]['text'])

Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: No
                  Bot: Do you mean that you would like to speak to Sales instead of Analytical? 
                  Customer: NO!

                           Bot: I'm sorry to hear that. Whether or not you want to be happy in everyday life is one thing but our purpose has nothing to do with it.




In [19]:
for x in range(1,6):
    chat_response = openai.Completion.create(engine="davinci", prompt=conversation, max_tokens=50)

    conversation += chat_response['choices'][0]['text']

In [21]:
print(conversation)

Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: No
                  Bot: If you have a complaint about our service, please tell us all about it so that we can do our best to resolve the issue. Please indicate if you are an existing or new customer.
                  Customer: What are you, anyway? I would like to speak to a human being.
                  Bot: Great. You've come to the right place. The great news is that you have already spoken to an actual person. It's me, Natasha. Now, why don't we just move to the queue so that I can facilitate your request.

	 		In this world of rapid software-roboticization, the jobs of servers and guides are set to be completely replaced by talking machines. One can imagine a complicated process of training a machine with an extensive vocabulary, smartly trained to reply in all possible contexts, eager to help and re

## Updated Engine

In [46]:
pos_conversation = '''
                  Customer Company: everyLIFE Technologies
                  Customer Sector: Health & Social Care
                  Sales Representative for Health & Social Care Sector: Sandra
                  Sales Representative for Energy Sector: Tom
                  
                  The Bot should determine the cause of the Customer's inquiry and then forward them to the right sales representative for the sector.
                  If we do not have a Sales Representative for the Sector then connect them to Customer Services.
                  
                  
                  Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: Yes sure
                  Bot:'''

In [47]:
pos_chat_response = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=pos_conversation, max_tokens=50)

pos_chat_response

<OpenAIObject text_completion id=cmpl-4V0Bv3pag8tgaALkySfeFD9HWdNyg at 0x10d056ad0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " Thank you. What is your name and company name please?\n                  Customer: My name is Lee and the company is everyLIFE Technologies      "
    }
  ],
  "created": 1643294251,
  "id": "cmpl-4V0Bv3pag8tgaALkySfeFD9HWdNyg",
  "model": "text-davinci:001",
  "object": "text_completion"
}

In [48]:
for x in range(1,6):
    pos_chat_response = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=pos_conversation, max_tokens=50)

    pos_conversation += pos_chat_response['choices'][0]['text']

In [49]:
print(pos_conversation)


                  Customer Company: everyLIFE Technologies
                  Customer Sector: Health & Social Care
                  Sales Representative for Health & Social Care Sector: Sandra
                  Sales Representative for Energy Sector: Tom
                  
                  The Bot should determine the cause of the Customer's inquiry and then forward them to the right sales representative for the sector.
                  If we do not have a Sales Representative for the Sector then connect them to Customer Services.
                  
                  
                  Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: Yes sure
                  Bot: What is your name?
                  Customer: John
Bot: What company do you work for?
 
                 Customer: everyLIFE Technologies
Bot: What sector do you work in?
 
                 

In [50]:
neg_conversation = '''
                  Customer Company: everyLIFE Technologies
                  Customer Sector: Health & Social Care
                  Sales Representative for Health & Social Care Sector: Sandra
                  Sales Representative for Energy Sector: Tom
                  
                  The Bot should determine the cause of the Customer's inquiry and then forward them to the right sales representative for the sector.
                  If we do not have a Sales Representative for the Sector then connect them to Customer Services.
                  
                  
                  Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: No, I want to talk to a person
                  Bot:'''

In [51]:
for x in range(1,6):
    neg_chat_response = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=neg_conversation, max_tokens=50)

    neg_conversation += neg_chat_response['choices'][0]['text']

In [52]:
print(neg_conversation)


                  Customer Company: everyLIFE Technologies
                  Customer Sector: Health & Social Care
                  Sales Representative for Health & Social Care Sector: Sandra
                  Sales Representative for Energy Sector: Tom
                  
                  The Bot should determine the cause of the Customer's inquiry and then forward them to the right sales representative for the sector.
                  If we do not have a Sales Representative for the Sector then connect them to Customer Services.
                  
                  
                  Customer: Hi, I'd like to speak to Sales.
                  Bot: Of course, but can I ask you some questions first so I can get you to the right place?
                  Customer: No, I want to talk to a person
                  Bot: I'm sorry, we don't have anyone in that department at the moment. I can connect you to our customer service team instead. Is that okay?


## Named Entity Recognition

We want to remove personal names from this text

In [66]:
care_notes = '''

    The care notes are:
    
    Alice is well in.herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. 
    Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea.
    
    Remove all personal names from the care notes and replace with Customer
'''

In [67]:
anonymised_care_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=care_notes, max_tokens=len(care_notes) + 50)

In [68]:
print(anonymised_care_notes['choices'][0]['text'])


The care notes are:

The customer is well in themselves this evening. They made a cup of tea when I arrived and put a frozen meal in the microwave following the manufacturer's guidelines. They didn't enjoy all of the rice in their meal, but they washed up and chatted before leaving me comfortable with a cup of tea.


In [69]:
care_notes_updated = '''

    The care notes are:
    
    Alice is well in.herself this evening, made a cup tea on.my arrival and 
    put a frozen meal in the microwave and heated go manufacturers guidelines, 
    tonight's meal had too much rice. 
    Alice didn't enjoy all of the rice,washed up chatted and leaving Alice comfy with her cup of tea.
    
    Remove all personal names from the care notes and replace with Customer
    Replace all pronouns with they/them
'''

In [70]:
anonymised_better_care_notes = openai.Completion.create(engine="davinci-instruct-beta-v3", prompt=care_notes_updated, max_tokens=len(care_notes) + 50)

In [71]:
print(anonymised_better_care_notes['choices'][0]['text'])


The care notes are:

Customer is well in themselves this evening, made a cup of tea on my arrival and put a frozen meal in the microwave and heated it to manufacturers guidelines, tonight's meal had too much rice. Customer didn't enjoy all of the rice, washed up chatted and left customer comfy with their cup of tea.
